In [1]:
#Get all the imports
import jax
import os
import sys

jax.config.update('jax_default_prng_impl', 'unsafe_rbg')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
os.environ["LIBTPU_INIT_ARGS"] = os.environ.get("LIBTPU_INIT_ARGS","") + " --xla_tpu_spmd_rng_bit_generator_unsafe=true"
print(f"Found {jax.device_count()} devices.")

from typing import Sequence
import datetime
from absl import app
from flax.linen import partitioning as nn_partitioning
from flax import linen as nn
import numpy as np
import optax
from tensorboardX import SummaryWriter

from layers import Transformer
import pyconfig
from input_pipeline import create_data_iterator_with_tokenizer
import max_utils
import checkpointing

import jax.numpy as jnp
from jax import random
from jax.sharding import PartitionSpec as P
from jax.sharding import Mesh

from jax.experimental.compilation_cache import compilation_cache as cc

from cloud_tpu_diagnostics import diagnostic
from cloud_tpu_diagnostics.configuration import debug_configuration
from cloud_tpu_diagnostics.configuration import diagnostic_configuration
from cloud_tpu_diagnostics.configuration import stack_trace_configuration

import max_utils
from jax.sharding import PartitionSpec as P
import max_logging
from tensorboardX import SummaryWriter

2023-12-06 21:05:57.640369: I external/xla/xla/pjrt/pjrt_api.cc:100] GetPjrtApi was found for tpu at /home/mazumdera/.local/lib/python3.10/site-packages/libtpu/libtpu.so
2023-12-06 21:05:57.640400: I external/xla/xla/pjrt/pjrt_api.cc:79] PJRT_Api is set for device type tpu
2023-12-06 21:05:57.640432: I external/xla/xla/pjrt/pjrt_api.cc:146] The PJRT plugin has PJRT API version 0.40. The framework PJRT API version is 0.40.
2023-12-06 21:06:00.316908: I external/xla/xla/pjrt/pjrt_c_api_client.cc:134] PjRtCApiClient created.


Found 4 devices.


2023-12-06 21:06:03.281930: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-06 21:06:03.546683: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 21:06:03.546738: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 21:06:03.547933: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 21:06:03.638436: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-06 21:06:03.638945: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
from flax.training import train_state

In [3]:
base_output_directory="base_output_directory=gs://mazumdera-test-bucket/maxtext/llama2/12062023/1"
base_num_decoder_layers="base_num_decoder_layers=32"
base_num_heads = "base_num_heads=32"
head_nums = "head_dim=128"
# activation_function="\"relu\""
# mlp_activations = f"mlp_activations=[{activation_function}]"
async_checkpointing="async_checkpointing=False" 
enable_dropout="enable_dropout=False"

#ici_data_parallelism=1
#ici_fsdp_parallelism=4

In [4]:
commandline_args = ["dummy", "/home/mazumdera/maxtext/MaxText/configs/base.yml","run_name=1xv4-8", "dcn_data_parallelism=1", "save_period=5","ici_data_parallelism=1","ici_tensor_parallelism=1","steps=20","enable_profiler=true","remat_policy=full","base_emb_dim=512", base_num_heads, head_nums,"vocab_size=50272", base_num_decoder_layers, "per_device_batch_size=0.5","enable_profiler=true", "base_mlp_dim=2048", base_output_directory, "max_predict_length=512", async_checkpointing, enable_dropout]# , mlp_activations]


In [6]:
pyconfig.initialize(commandline_args)
config = pyconfig.config

In [7]:
checkpoint_manager = checkpointing.create_orbax_checkpoint_manager(
      config.checkpoint_dir,
      config.enable_checkpointing,
      config.async_checkpointing,
      config.save_period,
  )

Creating checkpoint manager...
Checkpoint manager created!


In [8]:
# Initial PRNG Keys
init_rng, nextrng = random.split(random.PRNGKey(config.init_weights_seed), 2)

In [9]:
devices_array = max_utils.create_device_mesh(config)
mesh = Mesh(devices_array, config.mesh_axes)

Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)] (num_devices: 4)
Decided on mesh: [[[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)]]

  [[TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0)]]

  [[TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0)]]

  [[TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)]]]]


In [10]:
model = Transformer(config, mesh)

In [11]:
tx = optax.adamw(
      max_utils.create_learning_rate_schedule(config),
      b1=config.adam_b1,
      b2=config.adam_b2,
      eps=config.adam_eps,
      eps_root=config.adam_eps_root,
      weight_decay=config.adam_weight_decay,
  )
tx

GradientTransformationExtraArgs(init=<function chain.<locals>.init_fn at 0x7fbe54e2dbd0>, update=<function chain.<locals>.update_fn at 0x7fbe54e2de10>)

In [12]:
import max_utils
import functools

In [13]:
init_train_state_partial = functools.partial(max_utils.init_train_state, model, tx,
                                               config)

In [14]:
abstract_state = jax.eval_shape(init_train_state_partial, init_rng)
state_logical_annotations = nn.get_partition_spec(abstract_state)
unboxed_abstract_state = max_utils.unbox_logicallypartioned_trainstate(abstract_state)

In [15]:
abstract_state

TrainState(step=ShapeDtypeStruct(shape=(), dtype=int32), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fd17598c6d0>
    mesh = Mesh(device_ids=array([[[[0]],
    
            [[2]],
    
            [[1]],
    
            [[3]]]]), axis_names=('data', 'fsdp', 'sequence', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'ffn_layer1': {'kernel': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(512, 32, 2048), dtype=float32), names=('embed', 'layers', 'mlp'), mesh=None, rules=None)}, 'wi': {'kernel': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(512, 32, 2048), dtype=float32), names=('embed', 'layers', 'mlp'), mesh=None, rules=None)}, 'wo': {'kernel': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(2048, 32, 512), dtype=float32), names=('mlp', 'layers', 'embed'), mesh=None, rules=None)}}, 'post_self_attention_layer_norm': {'scale': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(512, 32), dtype=

In [16]:
state_logical_annotations

TrainState(step=PartitionSpec(), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fd17598c6d0>
    mesh = Mesh(device_ids=array([[[[0]],
    
            [[2]],
    
            [[1]],
    
            [[3]]]]), axis_names=('data', 'fsdp', 'sequence', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'ffn_layer1': {'kernel': PartitionSpec('embed', 'layers', 'mlp')}, 'wi': {'kernel': PartitionSpec('embed', 'layers', 'mlp')}, 'wo': {'kernel': PartitionSpec('mlp', 'layers', 'embed')}}, 'post_self_attention_layer_norm': {'scale': PartitionSpec('embed', 'layers')}, 'pre_self_attention_layer_norm': {'scale': PartitionSpec('embed', 'layers')}, 'self_attention': {'key': {'kernel': PartitionSpec('embed', 'layers', 'heads', 'kv')}, 'out': {'kernel': PartitionSpec('heads', 'layers', 'kv', 'embed')}, 'query': {'kernel': PartitionSpec('embed', 'layers', 'heads', 'kv')}, 'value': {'kernel': PartitionSpec('embed', 'layers', 

In [17]:
unboxed_abstract_state

TrainState(step=ShapeDtypeStruct(shape=(), dtype=int32), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fd17598c6d0>
    mesh = Mesh(device_ids=array([[[[0]],
    
            [[2]],
    
            [[1]],
    
            [[3]]]]), axis_names=('data', 'fsdp', 'sequence', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'ffn_layer1': {'kernel': ShapeDtypeStruct(shape=(512, 32, 2048), dtype=float32)}, 'wi': {'kernel': ShapeDtypeStruct(shape=(512, 32, 2048), dtype=float32)}, 'wo': {'kernel': ShapeDtypeStruct(shape=(2048, 32, 512), dtype=float32)}}, 'post_self_attention_layer_norm': {'scale': ShapeDtypeStruct(shape=(512, 32), dtype=float32)}, 'pre_self_attention_layer_norm': {'scale': ShapeDtypeStruct(shape=(512, 32), dtype=float32)}, 'self_attention': {'key': {'kernel': ShapeDtypeStruct(shape=(512, 32, 32, 128), dtype=float32)}, 'out': {'kernel': ShapeDtypeStruct(shape=(32, 32, 128, 512), dtype=float32)}, 'q

In [1]:
import jax.numpy as jnp

a = jnp.array([[1, 2,3,7], [4,5,6,8], [9,10,11,12], [13,14,15,16]])

a = jnp.array.[[1, 2,3,7], [4,5,6,8], [9,10,11,12], [13,14,15,16]])

# Swap the first and second dimensions
swapped_a = jnp.transpose(a, axes=(1, 0,2,3))

print(a.shape)
print(swapped_a)  # Output: [[1 3], [2 4]]
print(swapped_a.shape)

RuntimeError: Unable to initialize backend 'tpu': ABORTED: The TPU is already in use by process with pid 129629. Not attempting to load libtpu.so in this process. (set JAX_PLATFORMS='' to automatically choose an available backend)